# Injest data & Create local DB

- This notebook is used to read all the raw data files and combine them all into a nicely formatted sqlite3 database called `docs.db` .
- Some manual preprocessing was done on the index files and raw data files inbetween scraping and running this code to make sure they are all consistent.

### Notes
- The bottom of this notebook contains code for checking how well the merge went (ie. mismatches between index files and actual document files)
- After running this notebook I manually moved `docs.db` to the dropbox
- TODO: manual correction: move Ed Bastian file into Good lead examples - from nyt and update the index.csv files accordingly.


In [ ]:
cd ..

In [ ]:
# Jupyter magic
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# imports
import pandas as pd
from src import load_txt_files
from src import db_funcs

# Load docs & Create an index 
It's going to be a dataframe that stores:
- additional information about a document - interviewee, title, source, year, etc. 
- similarity scores
- file paths (temporary link)



get index to reflect the order of document, text, token lists

In [ ]:
# grab all the files from these directories and make some tokens
input_paths = [
        "./1_data/how_i_built_this/",
#         "../1_data/example_articles/",
        "./1_data/NYT_corner_office/",
        "./1_data/good_lead_articles/",
        "./1_data/current_transformational_client_articles/",
        "./1_data/Seeking_Alpha/",
    
        ]

files, docs, paths = load_txt_files.add_files_from_dir(input_paths)
df_index_a = load_txt_files.assemble_index_files(input_paths)
df_index_b = pd.DataFrame([files, paths, docs], index=['filename', 'group', 'text']).T


a = df_index_a.set_index('filename')
b = df_index_b.set_index("filename")

df_index = b.join(a, how='left')
df_index.shape, df_index_b.shape, df_index_a.shape

In [ ]:
sub_df = df_index.loc[df_index['group'] == "Seeking_Alpha"]
titles = []
for i, row in sub_df.iterrows():
    org = row['org']
    year = 2020 #row['year']
    title = f"{org} ({year})"
    titles.append(title)

df_index.loc[sub_df.index, 'year'] = 2020
df_index.loc[sub_df.index, 'title'] = titles

## save everything to local database

In [ ]:
# fname = './1_data/Seeking_Alpha/index.csv'

# d = pd.read_csv(fname, index_col=0)
# d.to_csv('sa_index.csv')
# d = d.drop(columns=['Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1', 'f2'])
# d['publication'] = 'earnings call'
# d = d.set_index('filename')
# d.index = [str(i).split('.txt')[0] for i in d.index]
# d.index.name = 'filename'
# d.to_csv(fname)
# print(len(d))
# d

In [ ]:
db_filename = 'docs.db'
con = db_funcs.create_connection(db_filename)
df_index.to_sql('docs', con=con)

### Check overlap between actual files and index.csv contents

In [ ]:
# len(df_index_a), len(df_index_b)
# files_not_in_indexed
# index_without_files

In [ ]:
files_not_in_indexed = list(set(df_index_b['filename']) - set(df_index_a['filename']))
index_without_files = list( set(df_index_a['filename']) - set(df_index_b['filename']))
overlap = list( set(df_index_a['filename']) | set(df_index_b['filename']))

print(f'{len(files_not_in_indexed)} files not referenced by an index.csv')
print(f'{len(index_without_files)} indexes missing a file')
print()
print(f'{len(overlap)} overlaps')

# Check if order accurate

In [ ]:
index_order = df_index.index.to_list()
file_order = df_index_b['filename'].to_list()
for i in range(len(df_index)):
    filename1 = index_order[i]
    filename2 = file_order[i]
    if filename1 != filename2:
        print('ERROR', i, filename1, filename2)

In [ ]:
## random spotcheck to see if name matches article
import random

def check_doc_index_match():

#     d = 'good_lead_articles'
#     subsample = df_index[df_index['group'] == d]
    subsample = df_index
    random_article = random.sample(list(subsample.index), 1)[0]
    i = df_index.index.get_loc(random_article)
    print(i, random_article)
    print(df_index.loc[random_article, 'name'])
    print('---')
    print(docs[i])
    
check_doc_index_match()

In [ ]:
# check for duplicate filenames
df_index_b[df_index_b['filename'].duplicated()]

# look at order mistmatches

In [ ]:
missing_index = df_index_b.set_index('filename').loc[files_not_in_indexed].sort_values(by='group')
# missing_index[missing_index['dir'] == "current_transformational_client_articles"]
missing_index

In [ ]:
df_index_a.set_index('filename').loc[index_without_files]